In [1]:
!pip install "unsloth[kaggle-notebook]"
!pip install pandas openpyxl
!pip install unsloth_zoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 106.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.2/272.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 15.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 MB 6.0 MB/s eta 0:00:00:00:0100:

In [2]:
import os
# Cấu hình sử dụng 2 GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import pandas as pd
from unsloth import FastLanguageModel
from datasets import Dataset, load_dataset, concatenate_datasets
from trl import SFTTrainer
from transformers import TrainingArguments
from huggingface_hub import login
import random
import openpyxl


try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    HF_TOKEN = user_secrets.get_secret("hf_token")
    login(token=HF_TOKEN)
    print("Đăng nhập Hugging Face thành công bằng Kaggle Secrets.")
except Exception as e:
    print(f" Lỗi khi lấy token từ Kaggle Secrets: {e}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-10-28 14:26:39.356947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761661599.595814      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761661599.658575      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
Đăng nhập Hugging Face thành công bằng Kaggle Secrets.


In [ ]:
model_name = "unsloth/Llama-3.1-8B-Instruct"
repo_name = "VyDat/llama3-8b-multitask-10k"

VSL_EXCEL_PATH = "/kaggle/input/vie-vsl/Corpus-Vie-VSL-10K.xlsx"
SIGNTOSPEAK_JSON_PATH = "/kaggle/input/data-vsl/SigntoSpeak.json"
CHAT_DATASET_NAME = "bkai-foundation-models/vi-alpaca"

MAX_SAMPLES_PER_DATASET = 10000
max_seq_length = 512
dtype = None

def format_vsl_data(sample):
    vietnamese_text = str(sample['vietnamese_text']).strip()
    vsl_structure = str(sample['vsl_structure']).strip()
    instruction_templates = [
        "Chuyển câu sau sang cấu trúc Ngôn ngữ ký hiệu Việt Nam (VSL): {}",
        "Hãy biểu diễn câu tiếng Việt này theo VSL: {}",
        "VSL cho câu '{}' là gì?",
        "Dịch sang ngôn ngữ ký hiệu: {}",
    ]
    user_prompt = random.choice(instruction_templates).format(vietnamese_text)
    
    conversation = [

        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": vsl_structure}
    ]
    return {"text": tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=False)}


def format_signtospeak_data(sample):
    vsl_text = str(sample['input']).strip()
    vietnamese_text = str(sample['target']).strip()
    instruction_templates = [
        "Dịch cấu trúc VSL sau sang câu tiếng Việt hoàn chỉnh: {}",
        "Câu VSL '{}' có nghĩa là gì trong tiếng Việt?",
        "Chuyển từ VSL sang tiếng Việt: {}",
        "Đây là một chuỗi VSL, hãy dịch nó: {}",
    ]
    user_prompt = random.choice(instruction_templates).format(vsl_text)
    
    conversation = [
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": vietnamese_text}
    ]
    return {"text": tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=False)}

def format_chat_data(sample):
    instruction = str(sample['instruction']).strip()
    input_text = str(sample['input']).strip()
    output_text = str(sample['output']).strip()
    if input_text:
        user_prompt = f"{instruction}\n\n{input_text}"
    else:
        user_prompt = instruction
        
    conversation = [
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": output_text}
    ]
    return {"text": tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=False)}

print("--- Bắt đầu tải và xử lý dữ liệu cho 3 tác vụ ---")

_, tokenizer = FastLanguageModel.from_pretrained(model_name, max_seq_length=max_seq_length)

datasets_list = []

try:
    df_vsl = pd.read_excel(VSL_EXCEL_PATH, header=None)
    df_vsl.columns = ['vietnamese_text', 'vsl_structure']
    df_vsl = df_vsl.dropna().sample(frac=1, random_state=42)
    vsl_dataset = Dataset.from_pandas(df_vsl).select(range(min(len(df_vsl), MAX_SAMPLES_PER_DATASET)))
    vsl_dataset = vsl_dataset.map(format_vsl_data, num_proc=1)
    datasets_list.append(vsl_dataset)
    print(f"Đã xử lý {len(vsl_dataset)} mẫu từ VSL (Vi -> VSL)")
except Exception as e:
    print(f"Lỗi khi xử lý VSL: {e}")

try:
    signtospeak_dataset = load_dataset('json', data_files=SIGNTOSPEAK_JSON_PATH, split='train')
    signtospeak_dataset = signtospeak_dataset.shuffle(seed=42).select(range(min(len(signtospeak_dataset), MAX_SAMPLES_PER_DATASET)))
    signtospeak_dataset = signtospeak_dataset.map(format_signtospeak_data, num_proc=1)
    datasets_list.append(signtospeak_dataset)
    print(f"Đã xử lý {len(signtospeak_dataset)} mẫu từ Sign-to-Speak (VSL -> Vi)")
except Exception as e:
    print(f"Lỗi khi xử lý Sign-to-Speak: {e}")


try:
    chat_dataset = load_dataset(CHAT_DATASET_NAME, split="train")
    chat_dataset = chat_dataset.shuffle(seed=42).select(range(min(len(chat_dataset), MAX_SAMPLES_PER_DATASET)))

    chat_dataset = chat_dataset.map(format_chat_data, remove_columns=chat_dataset.column_names, num_proc=4)
    datasets_list.append(chat_dataset)
    print(f"Đã xử lý {len(chat_dataset)} mẫu từ {CHAT_DATASET_NAME} (Chat)")
except Exception as e:
    print(f"Lỗi khi xử lý bộ Chat: {e}")

if not datasets_list:
    raise ValueError("Không có bộ dữ liệu nào được tải thành công! Vui lòng kiểm tra lại đường dẫn và code.")

final_dataset = concatenate_datasets(datasets_list).shuffle(seed=42)

final_dataset = final_dataset.filter(
    lambda x: len(tokenizer(x['text']).input_ids) <= max_seq_length
)
print(f"\nTổng số mẫu cuối cùng (3 tác vụ) sau khi gộp và lọc: {len(final_dataset)}")


print("\n--- Đang khởi tạo Model và Tokenizer ---")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=False,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=128,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=final_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=4,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        num_train_epochs=2,
        learning_rate=3e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs",
        save_strategy="steps",
        save_steps=200,
        save_total_limit=3,
        report_to="none",
    ),
)

print("\n Start")
torch.cuda.empty_cache()

trainer_stats = trainer.train()

print("\n Đang lưu và đẩy model lên Hugging Face Hub")

try:

    model.push_to_hub(repo_name, use_temp_dir=True)
    tokenizer.push_to_hub(repo_name, use_temp_dir=True)
    print(f"Training hoàn thành! Model đã được đẩy lên repo: https://huggingface.co/{repo_name}")
except Exception as e:
    print(f"Đã xảy ra lỗi khi đẩy model lên Hub: {e}")
    print("Đang thử lưu model xuống local tại './{repo_name}'")
    model.save_pretrained(repo_name)
    tokenizer.save_pretrained(repo_name)
    print(f"Đã lưu model và tokenizer tại './{repo_name}'")

--- Bắt đầu tải và xử lý dữ liệu cho 3 tác vụ ---
==((====))==  Unsloth 2025.10.11: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Đã xử lý 10000 mẫu từ VSL (Vi -> VSL)
Đã xử lý 10000 mẫu từ Sign-to-Speak (VSL -> Vi)
Đã xử lý 10000 mẫu từ bkai-foundation-models/vi-alpaca (Chat)

Tổng số mẫu cuối cùng (3 tác vụ) sau khi gộp và lọc: 29228

--- Đang khởi tạo Model và Tokenizer ---
==((====))==  Unsloth 2025.10.11: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


The model is already on multiple devices. Skipping the move to device specified in `args`.



Bắt đầu quá trình huấn luyện 3 tác vụ


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 29,228 | Num Epochs = 2 | Total steps = 1,828
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 45,088,768 of 1,280,903,168 (3.52% trained)


Step,Training Loss
25,2.264900
50,1.234000
75,1.037500
100,1.050000
